In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from skimage.transform import resize
import matplotlib.pyplot as plt
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')


2024-10-14 23:50:21.926072: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ModuleNotFoundError: No module named 'skimage'

In [ ]:
# Load the MNIST dataset
# (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
MNIST_train = pd.read_csv("mnist_train_100.csv", header=None)
MNIST_test = pd.read_csv("mnist_test_10.csv", header=None)

y_train = MNIST_train[0]
y_test = MNIST_test[0]

x_train = MNIST[1:]


# Combine train and test for uniform processing
x = np.concatenate((x_train, x_test), axis=0)
y = np.concatenate((y_train, y_test), axis=0)

# Normalize the pixel values to [0, 1]
x = x.astype('float32') / 255.0

# One-hot encode the labels
y = to_categorical(y, num_classes=10)

# Split back into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42, stratify=y)


In [ ]:
# Define the list of resolutions to test (from high to low)
resolutions = [28, 24, 20, 16, 14, 12, 10]


In [ ]:
def downscale_images(images, new_size):
    """
    Downscale a set of images to the specified size.

    Parameters:
    - images: numpy array of shape (num_samples, height, width)
    - new_size: int, the desired height and width

    Returns:
    - downscaled_images: numpy array of shape (num_samples, new_size, new_size)
    """
    downscaled_images = np.zeros((images.shape[0], new_size, new_size))
    for i in range(images.shape[0]):
        # Resize each image
        downscaled_images[i] = resize(images[i], (new_size, new_size), anti_aliasing=True)
    return downscaled_images


In [ ]:
def build_ann(input_dim):
    """
    Build and compile an ANN model.

    Parameters:
    - input_dim: int, the number of input features

    Returns:
    - model: compiled Keras model
    """
    model = keras.Sequential([
        layers.Dense(128, activation='relu', input_dim=input_dim),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [ ]:
# Dictionary to store accuracies for each resolution
accuracy_dict = {}

# Record the original accuracy for comparison
# First, evaluate the model on original resolution
original_size = 28
MNIST_train_orig = MNIST_train
x_test_orig = x_test

# Flatten the images for ANN input
MNIST_train_flat = MNIST_train_orig.reshape((MNIST_train_orig.shape[0], -1))
x_test_flat = x_test_orig.reshape((x_test_orig.shape[0], -1))

# Build and train the model on original resolution
print(f"\nTraining on original resolution: {original_size}x{original_size}")
model_orig = build_ann(input_dim=original_size*original_size)
history_orig = model_orig.fit(MNIST_train_flat, y_train,
                              epochs=10,
                              batch_size=128,
                              validation_split=0.1,
                              verbose=0)
loss_orig, acc_orig = model_orig.evaluate(x_test_flat, y_test, verbose=0)
print(f"Original Resolution Test Accuracy: {acc_orig*100:.2f}%")

# Store the original accuracy
accuracy_dict[original_size] = acc_orig


In [ ]:
# Iterate over the defined resolutions (excluding original)
for size in resolutions[1:]:
    print(f"\nProcessing resolution: {size}x{size}")
    
    # Downscale training and testing images
    MNIST_train_ds = downscale_images(MNIST_train, size)
    x_test_ds = downscale_images(x_test, size)
    
    # Flatten the images
    MNIST_train_ds_flat = MNIST_train_ds.reshape((MNIST_train_ds.shape[0], -1))
    x_test_ds_flat = x_test_ds.reshape((x_test_ds.shape[0], -1))
    
    # Build the model
    model = build_ann(input_dim=size*size)
    
    # Train the model
    history = model.fit(MNIST_train_ds_flat, y_train,
                        epochs=10,
                        batch_size=128,
                        validation_split=0.1,
                        verbose=0)
    
    # Evaluate the model
    loss, acc = model.evaluate(x_test_ds_flat, y_test, verbose=0)
    print(f"Test Accuracy at {size}x{size}: {acc*100:.2f}%")
    
    # Store the accuracy
    accuracy_dict[size] = acc


In [ ]:
# Display the accuracies
print("\nClassification Accuracies at Various Resolutions:")
for size in sorted(accuracy_dict.keys(), reverse=True):
    print(f"{size}x{size}: {accuracy_dict[size]*100:.2f}%")

# Plot the accuracies
plt.figure(figsize=(10,6))
sizes = sorted(accuracy_dict.keys(), reverse=True)
accs = [accuracy_dict[size]*100 for size in sizes]
plt.plot(sizes, accs, marker='o')
plt.title('MNIST Classification Accuracy vs. Image Resolution')
plt.xlabel('Image Resolution (Pixels)')
plt.ylabel('Accuracy (%)')
plt.gca().invert_xaxis()  # Higher resolutions on the left
plt.grid(True)
plt.show()


In [ ]:
Original Resolution Test Accuracy: 98.00%

Classification Accuracies at Various Resolutions:
28x28: 98.00%
24x24: 97.80%
20x20: 97.50%
16x16: 96.80%
14x14: 96.50%
12x12: 95.00%
10x10: 93.50%
